In [1]:
######################## 机器学习对数几率回归类 ##############################
# Author: 杨玉奇
# email: yangyuqi@sina.com
# url: https://github.com/jerryyyq/tf_algorithm_example
# copyright yangyuqi
# 著作权归作者 杨玉奇 所有。商业转载请联系作者获得授权，非商业转载请注明出处。
# date: 2017-09-13
###################################################################

from ML_Model import ML_Model
import tensorflow as tf
    
class ML_Sigma_Regression( ML_Model ):
    def __init__(self):
        ML_Model.__init__(self)

        self.__W = tf.Variable(tf.zeros([5, 1]), name = 'weights')
        self.__b = tf.Variable(0., name = 'bias')
    
        
    def combine_inputs(self, features):
        return tf.matmul(features, self.__W) + self.__b
    
    def inference(self, features):
        return tf.sigmoid( self.combine_inputs(features) )
    
    def loss(self, features, label):
        label_predicted = self.combine_inputs(features)
        return tf.reduce_mean( tf.nn.sigmoid_cross_entropy_with_logits(label_predicted, label) )
    
    
    def inputs(self):
        batch_size = 10
        passenger_id, survived, pclass, name, sex, age, sibsp, parch, ticket, fare, cabin, embarked = \
        self._read_csv(batch_size, ['Titanic_train.csv'], [[0.0], [0.0], [0], [''], [''], [0.0], [0.0], [0.0], [''], [0.0], [''], ['']])

        # return passenger_id, survived
        
        is_first_class = tf.to_float(tf.equal(pclass, [1]))
        is_second_class = tf.to_float(tf.equal(pclass, [2]))
        is_third_class = tf.to_float(tf.equal(pclass, [3]))

        gender = tf.to_float(tf.equal(sex, ['female']))

        features = tf.transpose(tf.pack([is_first_class, is_second_class, is_third_class, gender, age]))
        label = tf.reshape(survived, [batch_size, 1])
        return features, label 
        
        
    def train(self, loss):    
        learning_rate = 0.01
        return tf.train.GradientDescentOptimizer( learning_rate ).minimize( loss )

    
    def evaluate(self, features, label):
        predicted = tf.cast( self.inference(features) > 0.5, tf.float32 )
        self._echo_tensor( tf.reduce_mean(tf.cast(tf.equal(predicted, Y), tf.float32)) )

    
    def do_train(self, training_steps = 1000):
        print( 'do_train: start' )
        
        self._ML_Model__sess.run( tf.initialize_all_variables() )
    
        features, label = self.inputs()
    
        coord = tf.train.Coordinator() 
        threads = tf.train.start_queue_runners( sess = self._ML_Model__sess, coord = coord ) 

        try:
            total_loss = self.loss( features, label ) 
            train_op = self.train( total_loss ) 

            for step in range(training_steps):    # 实际训练闭环
                # print(total_loss)
                self._ML_Model__sess.run(train_op)
            
                # 查看训练过程损失递减
                if step % 10 == 0:
                    # print( str(step) + " loss: ", sess.run([total_loss]) )
                    self._echo_tensor( total_loss, 'step_' + str(step) + ' loss: ' )

            #print( str(training_steps) + " final loss: ", sess.run([total_loss]) )
            self._echo_tensor( total_loss, 'step_' + str(step) + ' final loss: ' )

            # 模型评估
            # self.evaluate( features, label ) 
            
    
        except tf.errors.OutOfRangeError:
            print( 'Done training -- epoch limit reached' )

        finally:
            coord.request_stop()        
            coord.join( threads )
            
        print( 'do_train: finish' )    
    
    
    
if __name__ == '__main__':
    one_ml = ML_Sigma_Regression()
    one_ml.do_train( 1 )

    

do_train: start
file_path =  ['/home/yangyuqi/work/tf_algorithm_example/Titanic_train.csv'] 

Tensor("ReaderRead:0", shape=(), dtype=string) Tensor("ReaderRead:1", shape=(), dtype=string)


InvalidArgumentError: Node 'gradients/Shape': Unknown input node Mean
	 [[Node: Reshape/shape = Const[dtype=DT_INT32, value=Tensor<type: int32 shape: [2] values: 10 1>]()]]
Caused by op 'Reshape/shape', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python3.5/dist-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/usr/local/lib/python3.5/dist-packages/tornado/ioloop.py", line 888, in start
    handler